In [ ]:
import random
from pathlib import Path
from math import sqrt, pi

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import seaborn as sns

%matplotlib inline

In [ ]:
## Load in all csv result files & combine them to one pandas table
wells = ['C' + '{0:02d}'.format(x) for x in range(2,5)] + \
        ['D' + '{0:02d}'.format(x) for x in range(2,5)] + \
        ['E' + '{0:02d}'.format(x) for x in range(2,5)] + \
        ['F' + '{0:02d}'.format(x) for x in range(2,5)]

wells_lamin = ['C' + '{0:02d}'.format(x) for x in range(2,4)] + \
              ['D' + '{0:02d}'.format(x) for x in range(2,4)] + \
              ['E' + '{0:02d}'.format(x) for x in range(2,4)] + \
              ['F' + '{0:02d}'.format(x) for x in range(2,4)]

# wells = ['B05', 'B07', 'E02', 'E05', 'F02', 'F05', 'F07', 'G02', 'G04', 'G05', 'G06', 'G07']
# wells = ['E02', 'E05', 'F02', 'F05', 'F07', 'G02', 'G04', 'G05', 'G06', 'G07']

base_path_features = '/Users/Joel/shares/workShareJoel/20200207-StainingTest7/featue-values/20200203-StainingTest7_p1_'
base_path_metadata = '/Users/Joel/shares/workShareJoel/20200207-StainingTest7/featue-values/20200203-StainingTest7_p1_'
base_path_popcon = '/Users/Joel/shares/workShareJoel/20200207-StainingTest7/popcon/20200203-StainingTest7_p1_'
base_path_edge = '/Users/Joel/shares/workShareJoel/20200207-StainingTest7/popcon/20200203-StainingTest7_p1_'

results = pd.DataFrame()

for well in wells:
    curr_features = pd.read_csv(base_path_features + well + '_ExpandedNuclei_feature-values.csv')
    curr_metadata = pd.read_csv(base_path_metadata + well + '_ExpandedNuclei_metadata.csv')
    curr_popcon = pd.read_csv(base_path_popcon + well + '_ExpandedNuclei_popcon.csv')
    curr_edge = pd.read_csv(base_path_edge + well + '_ExpandedNuclei_edge.csv')
    curr_well_tmp = curr_features.merge(curr_metadata, on = ['mapobject_id'])
    curr_well2 = curr_well_tmp.merge(curr_popcon, on = ['mapobject_id'])
    curr_well = curr_well2.merge(curr_edge, on = ['mapobject_id'])
    results = pd.concat([results, curr_well])

print(results.columns.values)


In [ ]:
sum_intensity_dapi = 'Intensity_sum_1_DAPI'
# Filter out cells that are thrown out by SVM
# correct_segment = (results['Missegmented'] == 0)
# non_mitotic = (results['Classification-439'] == 0)
non_border_cells = (results['is_border'] == 0)
gfp_good = (results['Intensity_sum_1_gfp_LaminB1'] > 500000) & (results['Intensity_mean_1_gfp_LaminB1'] > 150) & (results['Intensity_sum_1_ab_LaminB1'] < 4000000) & (results['Intensity_sum_2_ab_LaminB1'] < 4000000) & (results['Intensity_sum_3_ab_LaminB1'] < 4000000)

# Filter out cells that have very low DAPI signal. Mostly missegmented HEK cells
normal_dapi = (results[sum_intensity_dapi] < 2500000) & (results[sum_intensity_dapi] > 80000) & (results['Intensity_max_1_DAPI'] < 600) & (results['Intensity_mean_1_DAPI'] > 170)
# Filter out huge cells (missegmented cells)
normal_area = (results['Morphology_Area'] < 11000) & (results['Morphology_Area'] > 3500)
# no_dapi_debris = results['Intensity_max_dapi.rescaled_image'] < 1200

# Remove cells with staining artefacts in 568 channel

# no_568_artefact = ((results['Intensity_max_568.rescaled_image'] < 3500))
# no_647_artefact = ((results['Intensity_max_647.rescaled_image'] < 3000))

# Remove all cells that are in a boundary site (e.g. don't have a 3x3 neighborhood => potentially biased distance to edge)
non_boundary_sites = (results['well_pos_y'] != 0) & (results['well_pos_y'] != 11) & (results['well_pos_x'] != 0) & (results['well_pos_x'] != 11)

# Misclassified edges: Some cells were classified as being very close to an edge because of a 
# segmentation error (very big cell clump not being separated). They are in a very dense region, but classified as edge. Throw them out.
# no_edge_artefacts = ~((results['DistanceToEdge'] < 200) & (results['LocalDensity_Nuclei_500'] > 0.00045))

# Exclude sites with issues
bubble_1 = (results['well_name'] == 'C04') & (results['well_pos_y'] == 4) & (results['well_pos_x'] == 3)
bubble_2 = (results['well_name'] == 'E02') & (results['well_pos_y'] == 4) & (results['well_pos_x'] == 1)
bubble_3 = (results['well_name'] == 'F04') & (results['well_pos_y'] == 6) & (results['well_pos_x'] == 3)
bubble_4 = (results['well_name'] == 'F03') & (results['well_pos_y'] == 6) & (results['well_pos_x'] == 8)
bubble_5 = (results['well_name'] == 'F03') & (results['well_pos_y'] == 5) & (results['well_pos_x'] == 2)
# bubble_6 = (results['well_name'] == 'B04') & (results['well_pos_y'] == 6) & (results['well_pos_x'] == 8)
# bubble_7 = (results['well_name'] == 'B04') & (results['well_pos_y'] == 5) & (results['well_pos_x'] == 7)
# bubble_8 = (results['well_name'] == 'F06') & (results['well_pos_y'] == 1) & (results['well_pos_x'] == 0)
# gfp_issue_1 = (results['well_name'] == 'B04') & (results['well_pos_y'] == 4) & (results['well_pos_x'] == 5)
# gfp_issue_2 = (results['well_name'] == 'B04') & (results['well_pos_y'] == 5) & (results['well_pos_x'] == 5)
# gfp_issue_3 = (results['well_name'] == 'B06') & (results['well_pos_y'] == 7) & (results['well_pos_x'] == 1)
# gfp_issue_4 = (results['well_name'] == 'E03') & (results['well_pos_y'] == 4) & (results['well_pos_x'] == 7)
# good_sites = ~(bubble_1 | bubble_2 | bubble_3 | bubble_4 | bubble_5 | bubble_6 | bubble_7 | bubble_8 | gfp_issue_1 | gfp_issue_2 | gfp_issue_3 | gfp_issue_4)

good_sites = ~(bubble_1 | bubble_2 | bubble_3 | bubble_4 | bubble_5)

results_cleaned = results.loc[non_border_cells & non_boundary_sites & normal_dapi & normal_area & good_sites & gfp_good]

# results_cleaned = results.loc[correct_segment & non_mitotic & non_border_cells & normal_dapi & normal_area & no_dapi_debris & no_568_artefact & non_boundary_sites & no_edge_artefacts]
# results_cleaned = results.loc[non_border_cells]
print('Filtered out {} cells, kept {} ({:.2f}%)'.format(results.shape[0] - results_cleaned.shape[0], results_cleaned.shape[0], results_cleaned.shape[0]/results.shape[0] *100))

# results_cleaned['DAPI/568_sumIntensity'] = results_cleaned[sum_intensity_dapi] / results_cleaned[sum_Intensity_568]
# results_cleaned['DAPI/568_sumIntensity [log2]'] = np.log2(results_cleaned[sum_intensity_dapi] / results_cleaned[sum_Intensity_568])


In [ ]:
print(sum(non_border_cells)/results.shape[0])
# print(sum(correct_segment)/results.shape[0])
print(sum(normal_dapi)/results.shape[0])
print(sum(normal_area)/results.shape[0])
# print(sum(non_mitotic)/results.shape[0])
print(sum(non_boundary_sites)/results.shape[0])
print(sum(gfp_good)/results.shape[0])
print(sum(good_sites)/results.shape[0])

# edge_disagreements = (results_cleaned['DistanceToEdge'] < 100) & (results_cleaned['LocalDensity_Nuclei_500'] > 0.00045)
# print(results_cleaned.loc[edge_disagreements])
    

In [ ]:
# Function to filter by difference in DAPI over the cycles
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

def detect_linear_fit_outliers(results_df, feature1, feature2, curr_well_name, cutoff):
    '''
    Function makes a linear regression between the 2 features and returns a list of non-outliers
    
    Main usecase: Discard cells where DAPI varies too much over the cycles
    '''
    current_well = results_df[results_df['well_name'] == curr_well_name]

    # Fit the data(train the model)
    regression_model = LinearRegression()
    x = np.array(current_well[feature1]).reshape(-1, 1)
    y = np.array(current_well[feature2]).reshape(-1, 1)
    regression_model.fit(x, y)
    y_predicted = regression_model.predict(x)

    # Evaluate the model
    rmse = mean_squared_error(y, y_predicted)
    r2 = r2_score(y, y_predicted)
    residuals = y_predicted - y
    large_residuals = (np.abs(residuals) > cutoff).flatten()

    # Plot results
    # sns.distplot(residuals)
    # figure(figsize=(20, 15))
    # sns.scatterplot(x = current_well['Intensity_mean_1_DAPI'], y = current_well['Intensity_mean_2_DAPI'], hue = large_residuals, s=10)

    # Create output
    percentage_good_cells = np.sum(~large_residuals)/large_residuals.shape[0] *100
    print('In well {}, {} and {} correlated with an r2 of {}. Kept {}% of cells (with a cutoff of {})'.format(curr_well_name, feature1, feature2, round(r2, 3), round(percentage_good_cells,2), cutoff))
    mapobject_ids_non_outliers = current_well.loc[~large_residuals]['mapobject_id']
    return mapobject_ids_non_outliers

In [ ]:
# Filter by difference in DAPI over the cycles
# Some cells had imaging artefacts (most likely dirt under the plate) that influenced all channels for a cycle
# => Discard cells that vary in DAPI too much over the cycles
# Maybe needs to be done on a per well basis, as DAPI varies a bit between wells
# from matplotlib.pyplot import figure
# figure(figsize=(20, 15))
# well_class = pd.Series(results_cleaned['well_name'], dtype="category")
# sns.scatterplot(x = 'Intensity_mean_1_DAPI', y = 'Intensity_mean_2_DAPI', s=10, hue=well_class, data = results_cleaned)
cutoff = 35
moids_non_outliers_12 = pd.Series()
moids_non_outliers_13 = pd.Series()
moids_non_outliers_23 = pd.Series()

# Compare cycle 1 & 2
for curr_well in wells:
    moids_non_outliers_12 = pd.concat((moids_non_outliers_12, 
                                   detect_linear_fit_outliers(results_cleaned, feature1 = 'Intensity_mean_1_DAPI', 
                                                              feature2 = 'Intensity_mean_2_DAPI', 
                                                              curr_well_name = curr_well, cutoff = cutoff)))

# Compare cycle 1 & 3
for curr_well in wells:
    moids_non_outliers_13 = pd.concat((moids_non_outliers_13, 
                                   detect_linear_fit_outliers(results_cleaned, feature1 = 'Intensity_mean_1_DAPI', 
                                                              feature2 = 'Intensity_mean_3_DAPI', 
                                                              curr_well_name = curr_well, cutoff = cutoff)))
    
# Compare cycle 2 & 3
for curr_well in wells:
    moids_non_outliers_23 = pd.concat((moids_non_outliers_23, 
                                   detect_linear_fit_outliers(results_cleaned, feature1 = 'Intensity_mean_2_DAPI', 
                                                              feature2 = 'Intensity_mean_3_DAPI', 
                                                              curr_well_name = curr_well, cutoff = cutoff)))

df_moids_non_outliers_12 = pd.DataFrame(moids_non_outliers_12, columns=['mapobject_id'])
df_moids_non_outliers_13 = pd.DataFrame(moids_non_outliers_13, columns=['mapobject_id'])
df_moids_non_outliers_23 = pd.DataFrame(moids_non_outliers_23, columns=['mapobject_id'])
combined_non_outliers_tmp = df_moids_non_outliers_12.merge(df_moids_non_outliers_13, how='inner', on=['mapobject_id'])
combined_non_outliers = combined_non_outliers_tmp.merge(df_moids_non_outliers_23, how='inner', on=['mapobject_id'])

results_polished = results_cleaned.merge(combined_non_outliers, how='inner', on=['mapobject_id'])    


In [ ]:
# Make some quality control plots
# DAPI violin plots
sum_intensity_dapi = 'Intensity_sum_1_DAPI'
fig1, ax1 = plt.subplots(2, 1, figsize=(15, 20))
fig1.suptitle('DAPI (top) & Area (bottom)', fontsize = 20)
sns.violinplot(x = 'well_name', y = sum_intensity_dapi, data = results_polished, ax=ax1[0])
sns.violinplot(x = 'well_name', y = 'Morphology_Area', data = results_polished, ax=ax1[1])
# sns.violinplot(x = 'well_name', y = sum_intensity_dapi, data = results, ax=ax1[0])
# sns.violinplot(x = 'well_name', y = 'Morphology_Area', data = results, ax=ax1[1])

In [ ]:
# Select cells to show with cellfi
dapi_outliers = results_cleaned.loc[results_cleaned[sum_intensity_dapi] > 4500000]
print(dapi_outliers)
import tmclient
host="172.23.47.46"
port = 80
experimentName="20190723-StainingTest3"
username="joel"
password="123456"
mapobject="ExpandedNuclei-gfp"
client = tmclient.TmClient(host, port, username, password, experimentName)
cellfi = client.exhibit_mapobject(3870956, ['DAPI'], ooi=[mapobject], extra_margin=20) # , extra_margin=100
cellfi = list(cellfi)
for img in cellfi:
    plt.imshow(img, cmap='gray')

In [ ]:
# Plot local density measurements
fig1, ax1 = plt.subplots(2, 1, figsize=(15, 20))
fig1.suptitle('Local Density', fontsize = 20)
sns.violinplot(x = 'well_name', y = 'LocalDensity_ExpandedNuclei_500', data = results_polished, ax=ax1[0])
sns.violinplot(x = 'well_name', y = 'DistanceToEdge', data = results_polished, ax=ax1[1])

In [ ]:
# Save figures for all local density vs distance to edge plots
for well_name in wells_lamin:
    curr_well = results_polished.loc[(results_polished['well_name'] == well_name)]
    g = sns.jointplot(x = 'LocalDensity_ExpandedNuclei_500', y = 'DistanceToEdge', data = curr_well, xlim=(0, 0.00075), kind ='kde')
    g.savefig('20200210-StainingTest7_' + well_name + '_DensityVsEdge.png')

In [ ]:
# Calculate log ratios
results_polished['gfp/ab sumIntensity Cycle1 [log2]'] = np.log2(results_polished['Intensity_sum_1_gfp_LaminB1'] / results_polished['Intensity_sum_1_ab_LaminB1'])
results_polished['gfp/ab sumIntensity Cycle2 [log2]'] = np.log2(results_polished['Intensity_sum_1_gfp_LaminB1'] / results_polished['Intensity_sum_2_ab_LaminB1'])
results_polished['gfp/ab sumIntensity Cycle3 [log2]'] = np.log2(results_polished['Intensity_sum_1_gfp_LaminB1'] / results_polished['Intensity_sum_3_ab_LaminB1'])


# for well_name in wells:
#     curr_well = results_cleaned.loc[(results_cleaned['well_name'] == well_name)]
#     g = sns.jointplot(x = 'DistanceToEdge', y = 'gfp/ab sumIntensity [log2]', data = curr_well, s=5, xlim=(-500, 3500), ylim=(-2.0, 2.0), kind ='kde')
#     g.savefig('20191104-StainingTest4_gfpVSab_Rescaled_' + well_name + '.png')


In [ ]:
# Save plots for all distance to edge vs. log intensity ratio plots
for well_name in wells_lamin:
    curr_well = results_polished.loc[(results_polished['well_name'] == well_name)]
    g = sns.jointplot(x = 'DistanceToEdge', y = 'gfp/ab sumIntensity Cycle2 [log2]', data = curr_well, s=5, xlim=(-500, 3500), ylim=(-1.0, 1.0), kind ='kde')
    g.savefig('20200210-StainingTest7_gfpVSabCycle2_' + well_name + '.png')

In [ ]:
# Plot relationship DAPI/nanog (vs. density)
import scipy
# well_name = 'B04'
# for well_name in wells:
well_name = 'B05'
if well_name:
    curr_well = results_cleaned.loc[(results_cleaned['well_name'] == well_name)]

    # fig1, ax1 = plt.subplots(2, 1, figsize=(15, 20))
    # fig1.suptitle('DAPI/nanog Intensity Ratio', fontsize = 20)
    # sns.violinplot(x = 'well_name', y = 'DAPI/nanog_sumIntensity', data = results_cleaned, ax=ax1[0])

#     g = sns.jointplot(x = 'DistanceToEdge', y = 'gfp/ab sumIntensity [log2]', data = curr_well, xlim=(-500, 3500), ylim=(-2.0, 2.0), kind="reg")  # xlim=(0, 0.0025), ylim = (0.5, 4.5)
    g = sns.jointplot(x = 'DistanceToEdge', y = 'gfp/ab sumIntensity [log2]', data = curr_well, 
                      xlim=(-500, 3500), ylim=(-2.0, 2.0), kind="kde") # bw=.15


    
    # Calculate correlation between Density & DAPI/nanog
    density = np.array(curr_well['DistanceToEdge'])
    intensity_ratio = np.array(curr_well['gfp/ab sumIntensity [log2]'])
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(density, intensity_ratio)
    print(r_value*r_value)


    g.fig.suptitle(well_name + ', r2: ' + '{0:.2f}'.format(r_value*r_value) + ', slope: ' + '{0:.2f} * 10^-3'.format(slope*1000))

    g.savefig('20191104-StainingTest4_GFPvsAB_regressionScatter_' + well_name + '.png')


In [ ]:
# Calculate a score for the change in staining ratio.
# Linear scores like slope or r2 don't work, because the major difference happening first is the 
# distribution changing from banana shape to linear
# Median Ratio Shift (MRS): Comparing cells at the edge (dist to edge = 0) with cells 1250-1750 from the edge
# Calculating the median gfp/ab ratio for both and the shift between the 2 populations
results_polished['gfp/ab sumIntensity Cycle2 [log2]'] = np.log2(results_polished['Intensity_sum_1_gfp_LaminB1'] / results_polished['Intensity_sum_2_ab_LaminB1'])

mrs = []
# wells_of_interest = wells[:-5]
for wellname in wells_lamin:
    curr_well = results_polished.loc[(results_polished['well_name'] == wellname)]
    edge_cells = np.median(curr_well.loc[curr_well['DistanceToEdge'] == 0]['gfp/ab sumIntensity Cycle2 [log2]'])
    middle_cells = np.median(curr_well.loc[curr_well['DistanceToEdge'] > 1500]['gfp/ab sumIntensity Cycle2 [log2]'])
    mrs.append((wellname, wellname[0:1], int(wellname[1:]), middle_cells - edge_cells))
print(mrs)


In [ ]:
labels = ('well_name', 'row', 'col', 'mrs_score')
mrs_table = pd.DataFrame.from_records(mrs, columns=labels)
mrs_pivot = mrs_table.pivot('row', 'col', 'mrs_score')
g1 = sns.heatmap(mrs_pivot, annot=True, square=True)
mrs_figure = g1.get_figure()
mrs_figure.savefig('20200210-StainingTest7_MedianRatioShift.pdf')

In [ ]:
for well_name in wells_lamin:
    curr_well = results_polished.loc[(results_polished['well_name'] == well_name)]
    g = sns.jointplot(x = 'Intensity_mean_2_ab_LaminB1', y = 'Intensity_mean_3_ab_LaminB1', data = curr_well, s=5) #, xlim=(-500, 3500), ylim=(-1.0, 1.0) , kind ='kde'
    g.savefig('20200210-StainingTest7_laminCycle2VsCycle3_mean_' + well_name + '.png')

In [ ]:
r2s = []
import scipy
for wellname in wells_lamin:
    curr_well = results_polished.loc[(results_polished['well_name'] == wellname)]
    feature1 = 'Intensity_mean_2_ab_LaminB1'
    feature2 = 'Intensity_mean_3_ab_LaminB1'
    x = np.array(curr_well[feature1]).reshape(-1, 1)
    y = np.array(curr_well[feature2]).reshape(-1, 1)
    regression_model = LinearRegression()
    regression_model.fit(x, y)
    y_predicted = regression_model.predict(x)

    # Evaluate the model
    r2 = r2_score(y, y_predicted)
    
    r2s.append((wellname, wellname[0:1], int(wellname[1:]), r2))
    
print(r2s)


In [ ]:
wellname = 'B05'
curr_well = results_cleaned.loc[(results_cleaned['well_name'] == wellname)]
edge_cells = np.median(curr_well.loc[curr_well['DistanceToEdge'] == 0]['gfp/ab sumIntensity [log2]'])
middle_cells = np.median(curr_well.loc[curr_well['DistanceToEdge'] > 1500]['gfp/ab sumIntensity [log2]'])
print(edge_cells, middle_cells)

In [ ]:
# Local density mapped back on the images
img_width = 2560
img_height = 2160

density_img = np.zeros((img_width, img_height))

# Define site of interest
well_name = 'C03'
well_pos_y = 0
well_pos_x = 3
current_site_bool = (results['well_name'] == well_name) & (results['well_pos_y'] == well_pos_y)  & (results['well_pos_x'] == well_pos_x)

current_site = results.loc[current_site_bool]

# Get the segmentation mask from TissueMaps
import tmclient
host="172.23.47.46"
port = 80
experimentName="20191031-StainingTest4"
username="joel"
password="123456"
mapobject="ExpandedNuclei"
client = tmclient.TmClient(host, port, username, password, experimentName)

plateDownload = client.get_plates()
plate_name = plateDownload[0]['name']

segmentation_image = client.download_segmentation_image(mapobject, plate_name, well_name, well_pos_y, well_pos_x)
# plt.figure(figsize=(15,20))
# imgplot = plt.imshow(segmentation_image)

img_width = 2560
img_height = 2160

density_img = np.zeros((img_height, img_width))

# Loop through all labels
labels = current_site['label']

for label in labels:
    current_cell = current_site.loc[current_site['label'] == label]
    density_img[segmentation_image == label] = current_cell['DistanceToEdge']

plt.figure(figsize=(15,20))
imgplot = plt.imshow(density_img)


In [ ]:
img_width = 2560
img_height = 2160

density_img = np.zeros((img_height, img_width))

# Loop through all labels
labels = current_site['label']

for label in labels:
    current_cell = current_site.loc[current_site['label'] == label]
    density_img[segmentation_image == label] = current_cell['isEdge']

plt.figure(figsize=(15,20))
imgplot = plt.imshow(density_img)


In [ ]:
img_width = 2560
img_height = 2160

density_img = np.zeros((img_height, img_width))

# Loop through all labels
labels = current_site['label']

for label in labels:
    current_cell = current_site.loc[current_site['label'] == label]
    density_img[segmentation_image == label] = current_cell['Intensity_mean_dapi.rescaled_image']

plt.figure(figsize=(15,20))
imgplot = plt.imshow(density_img)

In [ ]:
# Calculate average spot counts per well 
# print(results_cleaned.groupby('well_name')['mRNA_Count'].median())
# print(results_cleaned.groupby('well_name')['mRNA_Pbody_Count'].mean())

In [ ]:
# Look at percentage of transcripts in P-bodies
# results_cleaned = results_cleaned.assign(percentage_pbody_2 = 0)
# cells_with_pbodies = results_cleaned['mRNA_Count'] >= 1
# results_cleaned.loc[cells_with_pbodies,'percentage_pbody'] = results_cleaned['mRNA_Pbody_Count'][cells_with_pbodies]/results_cleaned['mRNA_Count'][cells_with_pbodies]
# print(results_cleaned['percentage_pbody'].describe())


In [ ]:
# fig1, ax1 = plt.subplots(2, 1, figsize=(15, 20))
# fig1.suptitle('mRNA Spot counts', fontsize = 20)
# g = sns.violinplot(x = 'well_name', y = 'percentage_pbody', data = results_cleaned.loc[hela_C_row], ax=ax1[0])
# g.set(xticklabels=['HPRT', 'Lamp1', 'CCND1', 'HIF1A', 'dapB', 'no probe', 'SPEN', 'no probe'])
# h = sns.violinplot(x = 'well_name', y = 'percentage_pbody', data = results_cleaned.loc[hek_E_row], ax=ax1[1])
# h.set(xticklabels=['HPRT', 'Lamp1', 'CCND1', 'HIF1A', 'dapB', 'no probe', 'SPEN', 'no probe'])


In [ ]:
# Plot percentage P-bodies vs cellular features (area, #Pbodies total, DNA content, protein content)
# fig1, ax1 = plt.subplots(2, 2, figsize=(15, 15))
# fig1.suptitle('Percentage Pbodies vs. cell Features', fontsize = 20)
# SPEN_hek = results_cleaned['well_name'] == 'E09'
# sns.jointplot(x = 'Nuclei_Intensity_sum_DAPI', y = 'percentage_pbody', data = results_cleaned.loc[SPEN_hek,], s=10)
# sns.jointplot(x = 'Pbodies_Count', y = 'percentage_pbody', data = results_cleaned.loc[SPEN_hek,],s=10)
# sns.jointplot(x = 'Morphology_Area', y = 'percentage_pbody', data = results_cleaned.loc[SPEN_hek,],s=10)
# sns.jointplot(x = 'Intensity_mean_Succs', y = 'percentage_pbody', data = results_cleaned.loc[SPEN_hek,],s=10)

# CCND1_hela = results_cleaned['well_name'] == 'C05'
# g = sns.jointplot(x = 'Nuclei_Intensity_sum_DAPI', y = 'percentage_pbody', data = results_cleaned.loc[CCND1_hela,], s=10)
# sns.jointplot(x = 'Pbodies_Count', y = 'percentage_pbody', data = results_cleaned.loc[CCND1_hela,],s=10)
# sns.jointplot(x = 'Morphology_Area', y = 'percentage_pbody', data = results_cleaned.loc[CCND1_hela,],s=10)
# sns.jointplot(x = 'Intensity_mean_Succs', y = 'percentage_pbody', data = results_cleaned.loc[CCND1_hela,],s=10)

# lamp1_hela = results_cleaned['well_name'] == 'C04'
# g = sns.jointplot(x = 'Nuclei_Intensity_sum_DAPI', y = 'percentage_pbody', data = results_cleaned.loc[lamp1_hela,], s=10)
# sns.jointplot(x = 'Pbodies_Count', y = 'percentage_pbody', data = results_cleaned.loc[lamp1_hela,],s=10)
# sns.jointplot(x = 'Morphology_Area', y = 'percentage_pbody', data = results_cleaned.loc[lamp1_hela,],s=10)
# sns.jointplot(x = 'Intensity_mean_Succs', y = 'percentage_pbody', data = results_cleaned.loc[lamp1_hela,],s=10)


In [ ]:
# Save dataframe as csv
# results_cleaned.to_csv('20181203-bDNA-FISH8-results-cleaned_improved.csv')

Load data directly from TissueMaps for single wells

In [ ]:
# import tmclient
# host="172.23.47.46"
# port = 80
# experimentName="20181203-bDNA-FISH-8"
# username="joel"
# password="123456"
# mapobject="Cells"
# client = tmclient.TmClient(host, port, username, password, experimentName)

# plateDownload = client.get_plates()
# plate_name = plateDownload[0]['name']

# wells = ['C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'E03', 'E08']
# for well in wells:
#     print(well)
#     site_y = 0
#     for site_x in range(1,3):
#         feature_values = client.download_feature_values(mapobject, plate_name, well, site_y, site_x)
#         print(feature_values['mRNA_T1_Count_bDNA-FISH'].mean())
# #         print(feature_values['mRNA_Pbody_T1_Count'].mean())

# wells = ['C09', 'C10']
# for well in wells:
#     print(well)
#     site_y = 0
#     for site_x in range(1,3):
#         feature_values = client.download_feature_values(mapobject, plate_name, well, site_y, site_x)
#         print(feature_values['mRNA_T4_Count_bDNA-FISH'].mean()) 